<a href="https://colab.research.google.com/github/vsamarnathoffl/AI_Career_Coach_GEN_AI/blob/main/AI_Career_Coach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages
!pip install langchain-google-genai langchain faiss-cpu PyPDF2 --quiet
!pip install sentence-transformers --quiet
!pip install -U langchain-community


In [ ]:
# Step 2: Import modules
import os
import PyPDF2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from google.colab import files
import getpass


In [ ]:
from IPython.display import Markdown, display

In [ ]:
# Step 3: Setup Gemini API key
# Safer way: prompt for key without exposing it
GOOGLE_API_KEY = getpass.getpass("🔑 Enter your Google Gemini API key: ")

llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0,
)

In [ ]:
# Step 4: Setup text splitter and embeddings
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)

embeddings = HuggingFaceEmbeddings()


In [ ]:
# Step 5: Upload your resume (PDF)
uploaded = files.upload()
pdf_path = next(iter(uploaded))


In [ ]:
# Step 6: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

resume_text = extract_text_from_pdf(pdf_path)


In [ ]:
# Step 7: Prompt template for summarizing resume
resume_summary_template = """
Role: You are an AI Career Coach.

Task: Given the candidate's resume, provide a comprehensive summary that includes the following key aspects:
- Career Objective
- Skills and Expertise
- Professional Experience
- Educational Background
- Notable Achievements

Instructions:
Provide a concise summary of the resume, focusing on the candidate's skills, experience, and career trajectory. Ensure the summary is well-structured, clear, and highlights the candidate's strengths in alignment with industry standards.

Requirements:
{resume}
"""

resume_prompt = PromptTemplate(
    input_variables=["resume"],
    template=resume_summary_template,
)

resume_analysis_chain = LLMChain(llm=llm, prompt=resume_prompt)
resume_summary = resume_analysis_chain.run(resume=resume_text)

formatted_summary = "### Resume Summary\n\n" + resume_summary
display(Markdown(formatted_summary))


In [ ]:
# Step 8: Build a QA System on Resume
splitted_text = text_splitter.split_text(resume_text)
vectorstore = FAISS.from_texts(splitted_text, embeddings)
vectorstore.save_local("vector_index")

db = FAISS.load_local("vector_index", embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

career_qa_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an experienced AI Career Coach.

You are helping a job seeker by answering questions based on their resume and your knowledge.

Here is the relevant context from the candidate's resume:
{context}

Question:
{question}

Give a structured, actionable answer with advice and steps to improve the candidate’s career prospects.

If the resume context is insufficient, use your general career coaching expertise to provide helpful answers.

Response should be straightforward and shorter without unnecessary information.
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": career_qa_prompt_template}
)

In [ ]:
while True:
    query = input("Ask a question about your resume or career (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    response = qa_chain.invoke(query)
    display(Markdown(f"**You:** {query}"))
    display(Markdown(f"###AI Career Coach:\n\n{response['result']}"))
